# Stock Price Prediction Example

This notebook demonstrates how to use the Stock Price Prediction model to predict future stock prices.

## 1. Setup and Imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import pandas_datareader as pdr

# Add the project root to the path
sys.path.append('..')

# Import project modules
from src.model import create_model, train_model
from src.predict import predict_future
from src.utils import create_dataset, preprocess_data, calculate_metrics, plot_predictions

# Load environment variables
load_dotenv()

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 2. Fetch Stock Data

In [ ]:
# Set stock symbol and date range
symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2023-12-31'

# Get API key from environment variables
api_key = os.getenv("TIINGO_API_KEY")
if not api_key:
    raise ValueError("TIINGO_API_KEY not found in environment variables. Please add it to .env file.")

# Fetch data from Tiingo
df = pdr.get_data_tiingo(symbol, api_key=api_key, start=start_date, end=end_date)

# Reset index if multi-level
if isinstance(df.index, pd.MultiIndex):
    df = df.reset_index()

# Ensure column names are lowercase
df.columns = [col.lower() for col in df.columns]

# Display the first few rows
df.head()

## 3. Visualize the Data

In [ ]:
# Plot the closing price
plt.figure(figsize=(14, 7))
plt.plot(df['date'], df['close'], label='Close Price')
plt.title(f'{symbol} Stock Price (2020-2023)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

## 4. Preprocess the Data

In [ ]:
# Set parameters
look_back = 60  # Number of previous time steps to use as input
train_size = 0.8  # Proportion of data to use for training

# Preprocess data
X_train, Y_train, X_test, Y_test, scaler = preprocess_data(df['close'], look_back, train_size)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

## 5. Create and Train the Model

In [ ]:
# Create model
input_shape = (look_back, 1)
model = create_model(input_shape)

# Display model summary
model.summary()

In [ ]:
# Set training parameters
epochs = 50
batch_size = 32

# Train model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

## 6. Evaluate the Model

In [ ]:
# Plot training history
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions
train_predict_original = scaler.inverse_transform(train_predict)
Y_train_original = scaler.inverse_transform(Y_train.reshape(-1, 1))
test_predict_original = scaler.inverse_transform(test_predict)
Y_test_original = scaler.inverse_transform(Y_test.reshape(-1, 1))

# Calculate metrics
train_metrics = calculate_metrics(Y_train_original, train_predict_original)
test_metrics = calculate_metrics(Y_test_original, test_predict_original)

print("Training Metrics:")
for metric, value in train_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nTesting Metrics:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

## 7. Visualize Predictions

In [ ]:
# Create a dataframe with actual and predicted values
results_df = pd.DataFrame({
    'Actual': Y_test_original.flatten(),
    'Predicted': test_predict_original.flatten()
})

# Plot actual vs predicted
plt.figure(figsize=(14, 7))
plt.plot(results_df.index, results_df['Actual'], label='Actual')
plt.plot(results_df.index, results_df['Predicted'], label='Predicted', alpha=0.7)
plt.title(f'{symbol} Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

## 8. Predict Future Prices

In [ ]:
# Set the number of days to predict
future_days = 30

# Get the last sequence from the test data
last_sequence = X_test[-1:]

# Predict future values
future_predictions = []
current_sequence = last_sequence[0]

for _ in range(future_days):
    # Reshape for prediction
    current_sequence_reshaped = current_sequence.reshape(1, look_back, 1)
    
    # Predict next value
    next_pred = model.predict(current_sequence_reshaped)[0, 0]
    
    # Add to predictions
    future_predictions.append(next_pred)
    
    # Update sequence
    current_sequence = np.append(current_sequence[1:], next_pred)

# Convert to numpy array
future_predictions = np.array(future_predictions).reshape(-1, 1)

# Inverse transform
future_predictions_original = scaler.inverse_transform(future_predictions)

# Create date range for future predictions
last_date = pd.to_datetime(df['date'].iloc[-1])
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days)

# Create dataframe for future predictions
future_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted': future_predictions_original.flatten()
})

# Display future predictions
future_df.head(10)

In [ ]:
# Plot historical data and future predictions
plt.figure(figsize=(16, 8))

# Plot historical data
plt.plot(df['date'], df['close'], label='Historical Data', color='blue')

# Plot future predictions
plt.plot(future_df['Date'], future_df['Predicted'], label='Future Predictions', color='red', linestyle='--')

# Add vertical line to separate historical data and predictions
plt.axvline(x=last_date, color='green', linestyle='-', alpha=0.7, label='Prediction Start')

plt.title(f'{symbol} Stock Price Prediction for Next {future_days} Days')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 9. Save the Model

In [ ]:
# Save the model
model.save('../models/stock_prediction_model.h5')
print("Model saved to '../models/stock_prediction_model.h5'")

## 10. Conclusion

In this notebook, we've demonstrated how to:
1. Fetch historical stock data using the Tiingo API
2. Preprocess the data for LSTM model training
3. Create and train an LSTM model
4. Evaluate the model's performance
5. Make predictions on test data
6. Predict future stock prices
7. Visualize the results

The model's performance can be further improved by:
- Using more historical data
- Adding more features (e.g., technical indicators)
- Tuning hyperparameters
- Trying different model architectures
- Implementing ensemble methods